In [1]:
import re

import numpy as np

from IPython.display import display, Markdown
from pathlib import Path
from collections import defaultdict

In [2]:
CC_PATTERN = re.compile(r"(?<!^)(?=[A-Z])")

In [3]:
BIODIVERSITY_PATTERN = re.compile(r"The simulation resulted in a biodiversity of (\d+) unique species\.")
EXECUTION_PATTERN = re.compile(r"The simulation took:\n - initialisation: ([^\n]+)\n - execution: ([^\n]+)\n - cleanup: ([^\n]+)\n")

In [4]:
TIME_PATTERN = re.compile(r"(\d+(?:\.\d+)?)([^\d]+)")
TIME_UNITS = {
    "ns": 0.000000001,
    "µs": 0.000001,
    "ms": 0.001,
    "s": 1.0,
}

def parse_time(time_str):
    match = TIME_PATTERN.match(time_str)
    
    if match is None:
        return None
    
    return float(match.group(1)) * TIME_UNITS[match.group(2)]

In [5]:
DOMAIN_FILE_PATTERN = re.compile(r"limit\.([+\-]?(?:0|[1-9]\d*)(?:\.\d*)?(?:[eE][+\-]?\d+)?)\.o\d+\.\d+")

In [6]:
display(Markdown("# Scalability of the IsolatedLandscape Independent Algorithm variant"))

results = defaultdict(list)

for path in Path(".").iterdir():
    match = DOMAIN_FILE_PATTERN.match(path.name)

    if match is None:
        continue

    domain = int(float(match.group(1)))

    with open(path) as file:
        stdout = file.read()

    match = EXECUTION_PATTERN.search(stdout)
    if match is None:
        continue
    initialisation = parse_time(match.group(1))
    execution = parse_time(match.group(2))
    cleanup = parse_time(match.group(3))

    results[domain].append((0, initialisation, execution, cleanup))

results = { domain: [(np.median(s), np.mean(s), np.std(s)) for s in zip(*result)] for domain, result in results.items() }

domains, biodiversities, initialisations, executions, cleanups = zip(*sorted([domain] + rs for domain, rs in results.items()))

for domain, execution in zip(domains, executions):
    display(Markdown(f"* ${domain}$ individuals: ${np.round(execution[1], 2)}s \pm {np.round(execution[2], 2)}s$ per batch"))

# Scalability of the IsolatedLandscape Independent Algorithm variant

* $1000000000$ individuals: $16002.04s \pm 2750.39s$ per batch

* $10000000000$ individuals: $6101.79s \pm 2119.15s$ per batch

* $71000000000$ individuals: $3753.76s \pm 2128.84s$ per batch